<a href="https://colab.research.google.com/github/heyggun/bigdata_analysis_test/blob/main/DataManim_type2_Classification_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 작업형 2유형

## 분류 Classification

### 성인 건강검진 데이터

- 데이터 설명 : 2018년도 성인의 건강검 진데이터 (종속변수 : 흡연상태 1- 흡연, 0-비흡연 )

- x_train: https://raw.githubusercontent.com/Datamanim/datarepo/main/smoke/x_train.csv
- y_train: https://raw.githubusercontent.com/Datamanim/datarepo/main/smoke/y_train.csv
- x_test: https://raw.githubusercontent.com/Datamanim/datarepo/main/smoke/x_test.csv 
- x_label(평가용) : https://raw.githubusercontent.com/Datamanim/datarepo/main/smoke/y_test.csv

- 데이터 출처 :https://www.data.go.kr/data/15007122/fileData.do (참고, 데이터 수정)

In [1]:
import pandas as pd
import numpy as np


from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
x_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/smoke/x_train.csv')
y_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/smoke/y_train.csv')
x_test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/smoke/x_test.csv')
y_test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/smoke/y_test.csv')

In [3]:
display(x_train.head(3))
print(x_train.shape)

display(y_train.head(3))
print(y_train.shape)

display(x_test.head(3))
print(x_test.shape)

display(y_test.head(3))
print(y_test.shape)

,ID,성별코드,연령대코드(5세단위),신장(5Cm단위),체중(5Kg단위),허리둘레,시력(좌),시력(우),청력(좌),청력(우),...,LDL콜레스테롤,혈색소,요단백,혈청크레아티닌,(혈청지오티)AST,(혈청지오티)ALT,감마지티피,구강검진수검여부,치아우식증유무,치석
0,0,F,40,155,60,81.3,1.2,1.0,1.0,1.0,...,126.0,12.9,1.0,0.7,18.0,19.0,27.0,Y,0.0,Y
1,1,F,40,160,60,81.0,0.8,0.6,1.0,1.0,...,127.0,12.7,1.0,0.6,22.0,19.0,18.0,Y,0.0,Y
2,2,M,55,170,60,80.0,0.8,0.8,1.0,1.0,...,151.0,15.8,1.0,1.0,21.0,16.0,22.0,Y,0.0,N


(44553, 26)


,ID,흡연상태
0,0,0
1,1,0
2,2,1


(44553, 2)


,ID,성별코드,연령대코드(5세단위),신장(5Cm단위),체중(5Kg단위),허리둘레,시력(좌),시력(우),청력(좌),청력(우),...,LDL콜레스테롤,혈색소,요단백,혈청크레아티닌,(혈청지오티)AST,(혈청지오티)ALT,감마지티피,구강검진수검여부,치아우식증유무,치석
0,8,M,80,160,65,91.0,0.9,0.7,1.0,1.0,...,94.0,14.5,1.0,1.0,29.0,19.0,39.0,Y,0.0,N
1,17,M,30,165,90,98.0,1.5,1.5,1.0,1.0,...,165.0,15.6,1.0,0.9,20.0,37.0,34.0,Y,0.0,N
2,20,M,40,170,75,81.0,1.5,1.5,1.0,1.0,...,98.0,16.4,1.0,0.7,25.0,32.0,73.0,Y,0.0,N


(11139, 26)


,ID,흡연상태
0,8,1
1,17,0
2,20,1


(11139, 2)


In [4]:
obj = x_train.select_dtypes(include='object').columns
num = x_train.select_dtypes(include='number').columns

print(len(x_train.columns))
print(len(obj))
print(len(num))

display(x_train[obj].head(3))
display(x_train[num].head(3))


26
3
23


,성별코드,구강검진수검여부,치석
0,F,Y,Y
1,F,Y,Y
2,M,Y,N


,ID,연령대코드(5세단위),신장(5Cm단위),체중(5Kg단위),허리둘레,시력(좌),시력(우),청력(좌),청력(우),수축기혈압,...,트리글리세라이드,HDL콜레스테롤,LDL콜레스테롤,혈색소,요단백,혈청크레아티닌,(혈청지오티)AST,(혈청지오티)ALT,감마지티피,치아우식증유무
0,0,40,155,60,81.3,1.2,1.0,1.0,1.0,114.0,...,82.0,73.0,126.0,12.9,1.0,0.7,18.0,19.0,27.0,0.0
1,1,40,160,60,81.0,0.8,0.6,1.0,1.0,119.0,...,115.0,42.0,127.0,12.7,1.0,0.6,22.0,19.0,18.0,0.0
2,2,55,170,60,80.0,0.8,0.8,1.0,1.0,138.0,...,182.0,55.0,151.0,15.8,1.0,1.0,21.0,16.0,22.0,0.0


In [5]:
num

Index(['ID', '연령대코드(5세단위)', '신장(5Cm단위)', '체중(5Kg단위)', '허리둘레', '시력(좌)', '시력(우)',
       '청력(좌)', '청력(우)', '수축기혈압', '이완기혈압', '식전혈당(공복혈당)', '총콜레스테롤', '트리글리세라이드',
       'HDL콜레스테롤', 'LDL콜레스테롤', '혈색소', '요단백', '혈청크레아티닌', '(혈청지오티)AST',
       '(혈청지오티)ALT', '감마지티피', '치아우식증유무'],
      dtype='object')

In [6]:
x_train['치아우식증유무'].value_counts()

0.0    35040
1.0     9513
Name: 치아우식증유무, dtype: int64

In [7]:
y_train['흡연상태'].value_counts()

0    28201
1    16352
Name: 흡연상태, dtype: int64

- 치아우식증유무 컬럼 범주형으로 변경해줌

In [8]:
x_train['치아우식증유무'] = x_train['치아우식증유무'].apply(lambda x: 'Y' if x==0.0 else 'N')
x_test['치아우식증유무'] = x_test['치아우식증유무'].apply(lambda x: 'Y' if x==0.0 else 'N')

In [9]:
obj = x_train.select_dtypes(include='object').columns
num = x_train.select_dtypes(include='number').columns

print(obj)
print(num)

display(x_train[obj].head(3))
display(x_train[num].head(3))

Index(['성별코드', '구강검진수검여부', '치아우식증유무', '치석'], dtype='object')
Index(['ID', '연령대코드(5세단위)', '신장(5Cm단위)', '체중(5Kg단위)', '허리둘레', '시력(좌)', '시력(우)',
       '청력(좌)', '청력(우)', '수축기혈압', '이완기혈압', '식전혈당(공복혈당)', '총콜레스테롤', '트리글리세라이드',
       'HDL콜레스테롤', 'LDL콜레스테롤', '혈색소', '요단백', '혈청크레아티닌', '(혈청지오티)AST',
       '(혈청지오티)ALT', '감마지티피'],
      dtype='object')


,성별코드,구강검진수검여부,치아우식증유무,치석
0,F,Y,Y,Y
1,F,Y,Y,Y
2,M,Y,Y,N


,ID,연령대코드(5세단위),신장(5Cm단위),체중(5Kg단위),허리둘레,시력(좌),시력(우),청력(좌),청력(우),수축기혈압,...,총콜레스테롤,트리글리세라이드,HDL콜레스테롤,LDL콜레스테롤,혈색소,요단백,혈청크레아티닌,(혈청지오티)AST,(혈청지오티)ALT,감마지티피
0,0,40,155,60,81.3,1.2,1.0,1.0,1.0,114.0,...,215.0,82.0,73.0,126.0,12.9,1.0,0.7,18.0,19.0,27.0
1,1,40,160,60,81.0,0.8,0.6,1.0,1.0,119.0,...,192.0,115.0,42.0,127.0,12.7,1.0,0.6,22.0,19.0,18.0
2,2,55,170,60,80.0,0.8,0.8,1.0,1.0,138.0,...,242.0,182.0,55.0,151.0,15.8,1.0,1.0,21.0,16.0,22.0


데이터 전처리

In [10]:
# 범주형 label_encoding


def do_label_encoding(df):

  cols = ['성별코드', '구강검진수검여부', '치아우식증유무', '치석']

  for col in cols:
    Encoder = LabelEncoder()
    df[col] = Encoder.fit_transform(df[col])

  return df


x_train_l = do_label_encoding(x_train)
x_test_l = do_label_encoding(x_test)

display(x_train_l.head(2))
display(x_test_l.head(2))

,ID,성별코드,연령대코드(5세단위),신장(5Cm단위),체중(5Kg단위),허리둘레,시력(좌),시력(우),청력(좌),청력(우),...,LDL콜레스테롤,혈색소,요단백,혈청크레아티닌,(혈청지오티)AST,(혈청지오티)ALT,감마지티피,구강검진수검여부,치아우식증유무,치석
0,0,0,40,155,60,81.3,1.2,1.0,1.0,1.0,...,126.0,12.9,1.0,0.7,18.0,19.0,27.0,0,1,1
1,1,0,40,160,60,81.0,0.8,0.6,1.0,1.0,...,127.0,12.7,1.0,0.6,22.0,19.0,18.0,0,1,1


,ID,성별코드,연령대코드(5세단위),신장(5Cm단위),체중(5Kg단위),허리둘레,시력(좌),시력(우),청력(좌),청력(우),...,LDL콜레스테롤,혈색소,요단백,혈청크레아티닌,(혈청지오티)AST,(혈청지오티)ALT,감마지티피,구강검진수검여부,치아우식증유무,치석
0,8,1,80,160,65,91.0,0.9,0.7,1.0,1.0,...,94.0,14.5,1.0,1.0,29.0,19.0,39.0,0,1,0
1,17,1,30,165,90,98.0,1.5,1.5,1.0,1.0,...,165.0,15.6,1.0,0.9,20.0,37.0,34.0,0,1,0


In [11]:
# 수치형 변수 standardScaler

def do_standard_scaler(df):

  cols = ['연령대코드(5세단위)', '신장(5Cm단위)', '체중(5Kg단위)', '허리둘레', '시력(좌)', '시력(우)',
       '청력(좌)', '청력(우)', '수축기혈압', '이완기혈압', '식전혈당(공복혈당)', '총콜레스테롤', '트리글리세라이드',
       'HDL콜레스테롤', 'LDL콜레스테롤', '혈색소', '요단백', '혈청크레아티닌', '(혈청지오티)AST',
       '(혈청지오티)ALT', '감마지티피']

  for col in cols:
    Scaler = StandardScaler()
    df[col] = Scaler.fit_transform(df[col].values.reshape(-1,1))

  return df


x_train_s = do_standard_scaler(x_train_l)
x_test_s = do_standard_scaler(x_test_l)

display(x_train_s.head(3))
display(x_test_s.head(3))

,ID,성별코드,연령대코드(5세단위),신장(5Cm단위),체중(5Kg단위),허리둘레,시력(좌),시력(우),청력(좌),청력(우),...,LDL콜레스테롤,혈색소,요단백,혈청크레아티닌,(혈청지오티)AST,(혈청지오티)ALT,감마지티피,구강검진수검여부,치아우식증유무,치석
0,0,0,-0.348302,-1.049853,-0.4588,-0.083771,0.385691,-0.016643,-0.16212,-0.163501,...,0.267783,-1.100546,-0.214896,-0.823115,-0.430331,-0.254602,-0.257586,0,1,1
1,1,0,-0.348302,-0.506284,-0.4588,-0.116105,-0.433770,-0.835046,-0.16212,-0.163501,...,0.292210,-1.228353,-0.214896,-1.265418,-0.220763,-0.254602,-0.435020,0,1,1
2,2,1,0.892501,0.580854,-0.4588,-0.223885,-0.433770,-0.425844,-0.16212,-0.163501,...,0.878471,0.752662,-0.214896,0.503794,-0.273155,-0.349076,-0.356161,0,1,0


,ID,성별코드,연령대코드(5세단위),신장(5Cm단위),체중(5Kg단위),허리둘레,시력(좌),시력(우),청력(좌),청력(우),...,LDL콜레스테롤,혈색소,요단백,혈청크레아티닌,(혈청지오티)AST,(혈청지오티)ALT,감마지티피,구강검진수검여부,치아우식증유무,치석
0,8,1,2.994013,-0.503176,-0.061759,0.980552,-0.241136,-0.641966,-0.16175,-0.165223,...,-0.505678,-0.079453,-0.217429,0.572159,0.144168,-0.285308,-0.010284,0,1,0
1,17,1,-1.172693,0.041569,1.890367,1.736744,1.004215,1.043663,-0.16175,-0.165223,...,1.231133,0.624321,-0.217429,0.077646,-0.297159,0.369629,-0.113340,0,1,0
2,20,1,-0.339351,0.586313,0.719092,-0.099721,1.004215,1.043663,-0.16175,-0.165223,...,-0.407829,1.136156,-0.217429,-0.911378,-0.051977,0.187702,0.690494,0,1,0


In [12]:
# 모델링

X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(x_train_s.drop(columns=['ID']), y_train['흡연상태'],
                                                            test_size=0.3, 
                                                            random_state=1234)

print(X_train_m.shape)
print(X_test_m.shape)
print(y_train_m.shape)
print(y_test_m.shape)

(31187, 25)
(13366, 25)
(31187,)
(13366,)


In [13]:

model = RandomForestClassifier()
model.fit(X_train_m, y_train_m)
prediction = model.predict_proba(X_test_m)[:,1]

In [14]:
from sklearn.metrics import roc_auc_score, f1_score

In [15]:
roc_auc_score(y_test_m, prediction)

0.8333762252142041

In [16]:
model.fit(x_train_s.drop(columns=['ID']), y_train['흡연상태'])
pred = model.predict_proba(x_test_s.drop(columns=['ID']))[:,1]

roc_auc_score(y_test['흡연상태'], pred)

0.9999187008992574

In [16]:
# f1 score 확인


class_pred = model.predict(x_test_s.drop(columns=['ID']))
f1_score(y_test['흡연상태'], class_pred, )